# 1. Carga de datos

In [1]:
# Librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Estilos
sns.set(style='whitegrid')

In [16]:
df = pd.read_csv('../datos/crudos/Life Expectancy Data.csv') # Lee los datos
df.columns = df.columns.str.strip() # Quita los espacios de los nombres de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10  BMI                              2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

In [46]:
# Revisamos los valores únicos de la columna Country y contamos los registros de cada uno
country = df.Country.value_counts()
country.head(10)

Country
Afghanistan         16
Peru                16
Nicaragua           16
Niger               16
Nigeria             16
Norway              16
Oman                16
Pakistan            16
Panama              16
Papua New Guinea    16
Name: count, dtype: int64

In [47]:
# Tomamos solo los países con al menos 16 registros
df = df.groupby('Country').filter(lambda x: len(x) >= 16)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2928 entries, 0 to 2937
Data columns (total 23 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2928 non-null   object 
 1   Year                             2928 non-null   int64  
 2   Status                           2928 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2928 non-null   int64  
 6   Alcohol                          2735 non-null   float64
 7   percentage expenditure           2928 non-null   float64
 8   Hepatitis B                      2375 non-null   float64
 9   Measles                          2928 non-null   int64  
 10  BMI                              2896 non-null   float64
 11  under-five deaths                2928 non-null   int64  
 12  Polio                    

In [44]:
df['nulos'] = df.isnull().sum(axis=1)
nulos_country = df.groupby('Country')['nulos'].sum().sort_values(ascending=False)
nulos_country = pd.DataFrame(nulos_country)
nulos_country.head(10)

,nulos
Country,
South Sudan,122
Somalia,91
Democratic People's Republic of Korea,84
United Kingdom of Great Britain and Northern Ireland,81
Democratic Republic of the Congo,73
Czechia,68
United Republic of Tanzania,68
Côte d'Ivoire,67
Republic of Moldova,66
